In [ ]:
import matplotlib
matplotlib.use('Qt5Agg')#for visbrain to work in jupyterlab/notebook
from header import *
import visbrain
from visbrain import Brain, Colorbar
from visbrain.objects import BrainObj, ColorbarObj, SceneObj, SourceObj
import xarray as xr

In [ ]:
task = 'SMEG'
path = op.join(Analysis_path, task, 'meg', 'Stats')
test_key = 'N_vs_E+RS'
sfreq = 200
start = 0.3
stop = 0.5
filename = op.join(path, '*-'+test_key+'-*'+str(start)+'_'+str(stop)+'*fsaverage*')
files = glob.glob(filename)

In [ ]:
pv_lh = np.loadtxt(files[0])
data_lh = np.loadtxt(files[1])

vertices = np.arange(data_lh.shape[1])

pv_rh = np.loadtxt(files[2])
data_rh = np.loadtxt(files[3])

In [ ]:
if np.where(pv_lh<0.05)[0].any():
    t_signif_lh = np.unique(np.where(pv_lh<0.05)[0], return_counts=True)
    best_t_lh = t_signif_lh[0][t_signif_lh[1].argsort()][::-1]
    print(t_signif_lh)
    print(best_t_lh)
else:
    warnings.warn('\nNo significant cluster for left hemisphere.\n')
    best_t_lh = [None]

In [ ]:
if np.where(pv_rh<0.05)[0].any():
    t_signif_rh = np.unique(np.where(pv_rh<0.05)[0], return_counts=True)
    best_t_rh = t_signif_rh[0][t_signif_rh[1].argsort()][::-1]
    print(t_signif_rh)
    print(best_t_rh)
else:
    warnings.warn('\nNo significant cluster for right hemisphere.\n')
    best_t_rh = [None]

In [ ]:
t = int((best_t_rh[0] if best_t_rh[0] is not None else best_t_lh[0]))
#t = int(best_t_rh[8])
#t = int(best_t_lh[0])
print('t =', (t+start*sfreq)/sfreq)
lh = np.where(pv_lh[t]<0.05)
rh = np.where(pv_rh[t]<0.05)

In [ ]:
cmin = np.min(np.concatenate([data_lh[t][lh], data_rh[t][rh]]))
cmax = np.max(np.concatenate([data_lh[t][lh], data_rh[t][rh]]))
clim = (-np.max(np.abs([cmin, cmax])), np.max(np.abs([cmin, cmax])))

In [ ]:
b_obj = BrainObj('inflated', translucent=False, hemisphere='both')#, sulcus=True)

#b_obj.add_activation(data=data_lh[t], vertices=vertices, hemisphere='left', cmap='bwr', clim=clim)
#b_obj.add_activation(data=data_rh[t], vertices=vertices, hemisphere='right', cmap='bwr', clim=clim)
#b_obj.add_activation(data=data_lh[t][lh], vertices=vertices[lh], hemisphere='left', cmap='bwr', clim=clim)
#b_obj.add_activation(data=data_rh[t][rh], vertices=vertices[rh], hemisphere='right', cmap='bwr', clim=clim)
#b_obj.add_activation(data=np.where(pv_lh[t]>=0.05, -np.inf, data_lh[t]), hide_under=clim[0], vertices=vertices, hemisphere='left', cmap='bwr', clim=clim)
#b_obj.add_activation(data=np.where(pv_lh[t]>=0.05, -np.inf, data_rh[t]), hide_under=clim[0], vertices=vertices, hemisphere='right', cmap='bwr', clim=clim)
b_obj.add_activation(data=np.where(pv_lh[t]<0.05, data_lh[t], 0), vertices=vertices, hemisphere='left', cmap='bwr', clim=clim)
b_obj.add_activation(data=np.where(pv_rh[t]<0.05, data_lh[t], 0), vertices=vertices, hemisphere='right', cmap='bwr', clim=clim)

vb = Brain(brain_obj=b_obj)
#cb = Colorbar(vmin=clim[0], vmax=clim[-1], cmap='bwr', ndigits=4, cblabel='T stat')

In [ ]:
vb.show()
vb.cbar_select('brain')
vb.cbar_control('brain', cblabel='{} (t = {} s)'.format(test_key, (t+start*sfreq)/sfreq))
vb.rotate(custom=(180, -45))
#cb.show()
#sc.preview()

In [ ]:
vb.screenshot(op.join(path,test_key+'_t_'+str((t+start*sfreq)/sfreq)+'.png'))

In [ ]:
t_signif_rh